#### 📂 Practical Lab 1

#### Streaming Data for Predictive Maintenance with Linear Regression-Based Alerts

In the data_stream_visualization workshop, the robot current data were ingested from the given CSV file and 150 rows were inserted into the database. 

In this workshop, all those rows will be retreived from the database and be used for Predictive Maintenance with Linear Regression-Based Alerts

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
from IPython.display import clear_output, display

from sqlalchemy import create_engine
from urllib.parse import quote_plus

import importlib

import modules.DBManager
importlib.reload(modules.DBManager)


# from sklearn import linear_model
# from sklearn.metrics import mean_squared_error, mean_absolute_error

<module 'modules.DBManager' from 'c:\\Users\\MOSTAFA\\Desktop\\Foundations of ML Frameworks\\Practical Lab 1\\Predictive-Maintenance-with-Linear-Regression\\modules\\DBManager.py'>

In [ ]:
# 🔐 Neon DB Config - Mostafa's db
DB_CONFIG = {
    "host": "ep-spring-surf-adwzqkg9-pooler.c-2.us-east-1.aws.neon.tech",
    "database": "neondb",
    "user": "neondb_owner",
    "password": "npg_2DdhmHi4GxFp",
    "port": "5432",
    "sslmode": "require",
}

dbmanager = modules.DBManager.DBManager(DB_CONFIG, columns=[])
data = dbmanager.fetch_all()

print(f"Total records in the database: {len(data)}")




🔍 Fetching all records from the database...
Total records in the database: 150
